multiple territorial units, multiple variables

Subject:

https://bdl.stat.gov.pl/api/v1/variables?subject-id=P3155&page-size=100

Przeciętne ceny skupu owoców i warzyw (dane półroczne)

Average procurement prices of fruits and vegetables (half-yearly data)

In [ ]:
import pandas as pd
import requests

In [ ]:
first_page = requests.get('https://bdl.stat.gov.pl/api/v1/variables?subject-id=P3155&page-size=100', timeout=7)
first_page = first_page.json()
first_page['results'][0]

{'id': 283913,
 'level': 3,
 'measureUnitId': 4,
 'measureUnitName': 'zł',
 'n1': 'I półrocze',
 'n2': 'kapusta za 1 kg',
 'subjectId': 'P3155'}

In [ ]:
last_link=first_page['links']['last']
last_page_num=int(last_link[last_link.find("page=")+5:] )
last_page_num

0

In [ ]:
# only one page
# 60 records

In [ ]:
"""
var_list=[]
for result in first_page['results']:
    if result['n1']=='rok':
        var_list.append(result)
"""
var_list=[result for result in first_page['results'] if result['n1']=='rok']

In [ ]:
var_list[1]

{'id': 283912,
 'level': 3,
 'measureUnitId': 4,
 'measureUnitName': 'zł',
 'n1': 'rok',
 'n2': 'cebula za 1 kg',
 'subjectId': 'P3155'}

In [ ]:
len(var_list)

20

In [ ]:
data_dict={}
unit_level=2 # województwa
for x in var_list:
    var_id=x['id']
    var_name=x['n2']
    url=f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{var_id}?page-size=100&unit-level={unit_level}&'
    for year in range(2011,2020+1):
        url+=f'year={year}&'
    try:
        page = requests.get(url, timeout=7)
    except:
        print("request error,",x)
        break
    page = page.json()
    data_list = [(result['name'], v['year'], v['val'])  for result in page['results'] for v in result['values']]
    data_dict[var_name]=data_list


In [ ]:
# view the variables
data_dict.keys()

dict_keys(['kapusta za 1 kg', 'cebula za 1 kg', 'marchew za 1 kg', 'buraki za 1 kg', 'ogórki za 1 kg', 'pomidory za 1 kg', 'kalafiory za 1 szt', 'inne warzywa za 1 kg', 'jabłka za 1 kg', 'gruszki za 1 kg', 'śliwki za 1 kg', 'brzoskwinie za 1 kg', 'wiśnie za 1 kg', 'czereśnie za 1 kg', 'truskawki za 1 kg', 'maliny za 1 kg', 'porzeczki za 1 kg', 'agrest za 1 kg', 'orzechy włoskie i laskowe za 1 kg', 'inne owoce za 1 kg'])

In [ ]:
# example values for one variable
data_dict['kapusta za 1 kg'][:3]

[('MAŁOPOLSKIE', '2012', 0.57),
 ('MAŁOPOLSKIE', '2013', 0.72),
 ('MAŁOPOLSKIE', '2014', 0.59)]

In [ ]:
# example dataframe (and a template for dataframe with all variables)
df=pd.DataFrame(data_dict['kapusta za 1 kg'], columns=['województwo', 'rok', 'kapusta za 1 kg'])
df

,województwo,rok,kapusta za 1 kg
0,MAŁOPOLSKIE,2012,0.57
1,MAŁOPOLSKIE,2013,0.72
2,MAŁOPOLSKIE,2014,0.59
3,MAŁOPOLSKIE,2015,0.86
4,MAŁOPOLSKIE,2016,0.76
...,...,...,...
123,MAZOWIECKIE,2015,1.03
124,MAZOWIECKIE,2016,1.17
125,MAZOWIECKIE,2017,0.86
126,MAZOWIECKIE,2018,0.94


In [ ]:
# joining remaining variables

for var_name in data_dict:
    if var_name=="kapusta za 1 kg":
        continue
    temp_df=pd.DataFrame(data_dict[var_name], columns=['województwo', 'rok', var_name])
    df=pd.merge(df, temp_df, how='left', on=['województwo','rok'])

In [ ]:
# whole dataset
df

,województwo,rok,kapusta za 1 kg,cebula za 1 kg,marchew za 1 kg,buraki za 1 kg,ogórki za 1 kg,pomidory za 1 kg,kalafiory za 1 szt,inne warzywa za 1 kg,jabłka za 1 kg,gruszki za 1 kg,śliwki za 1 kg,brzoskwinie za 1 kg,wiśnie za 1 kg,czereśnie za 1 kg,truskawki za 1 kg,maliny za 1 kg,porzeczki za 1 kg,agrest za 1 kg,orzechy włoskie i laskowe za 1 kg,inne owoce za 1 kg
0,MAŁOPOLSKIE,2012,0.57,0.55,0.76,0.45,2.23,3.38,0.93,1.67,0.70,1.92,1.64,2.70,2.82,4.81,4.70,2.94,1.69,3.43,18.29,1.77
1,MAŁOPOLSKIE,2013,0.72,0.60,0.78,0.64,2.05,2.39,0.96,1.64,0.99,1.42,1.02,2.15,2.10,4.80,1.80,5.28,1.61,3.29,14.18,6.10
2,MAŁOPOLSKIE,2014,0.59,0.70,0.60,0.54,1.85,2.79,1.03,1.70,0.50,1.33,0.91,1.91,1.31,4.38,2.71,4.69,1.28,2.87,15.13,2.03
3,MAŁOPOLSKIE,2015,0.86,0.85,0.97,0.61,2.25,2.71,1.10,2.03,0.74,1.78,1.45,1.31,2.39,5.52,2.00,8.62,1.26,1.70,28.43,4.21
4,MAŁOPOLSKIE,2016,0.76,0.78,0.77,0.55,1.88,2.99,1.10,1.92,0.58,1.73,1.25,2.33,1.20,6.71,2.97,3.94,1.49,1.11,15.13,4.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,MAZOWIECKIE,2015,1.03,0.74,0.61,0.47,2.19,2.58,1.20,1.78,0.84,1.97,1.53,2.76,1.71,5.56,2.34,8.86,0.73,2.09,24.46,3.90
124,MAZOWIECKIE,2016,1.17,0.64,0.64,0.62,1.62,2.22,1.24,1.77,0.62,1.94,1.16,2.13,1.37,4.04,2.71,5.47,1.19,1.31,7.97,3.18
125,MAZOWIECKIE,2017,0.86,0.46,0.38,0.60,1.93,2.59,1.28,1.47,1.08,2.60,2.64,1.69,3.32,10.76,4.47,4.84,1.73,3.42,17.53,3.86
126,MAZOWIECKIE,2018,0.94,0.93,0.43,0.48,2.12,2.64,1.20,1.48,0.70,1.95,0.81,1.76,1.22,4.13,3.66,3.46,1.20,1.28,7.90,5.02


In [ ]:
# save data

path='' 

df.to_csv(path+"ceny_skupu.csv", index=False)